## Q1. 겨울 시장은 투자 가치가 있는가? 🤔 
**가설:** 겨울에도 선크림 수요가 존재하며, 여름 대비 일정 비율 이상을 차지한다.

**검증 방법:**
- 계절별 검색량 비교
- 통계적 유의성 검정
- 연도별 트렌드 분석

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
import plotly.graph_objects as go
import plotly.express as px
from plotly.subplots import make_subplots

In [2]:
# 한글 폰트 설정
plt.rcParams['font.family'] = 'Malgun Gothic'  # Windows
plt.rcParams['axes.unicode_minus'] = False

In [3]:
df_trend = pd.read_csv("../data/01_search_trend_monthly.csv")
df_trend['date'] = pd.to_datetime(df_trend['date'])
df_trend['month'] = df_trend['date'].dt.month
df_trend['year'] = df_trend['date'].dt.year
df_trend['season'] = df_trend['month'].apply(
    lambda x: '봄' if 3 <= x <= 5 else
             '여름' if 6 <= x <= 8 else
             '가을' if 9 <= x <= 11 else '겨울'
)


In [4]:
# %%
# 계절별 평균 검색량
season_avg = df_trend.groupby('season')['선크림'].mean().sort_values(ascending=False)

print("📊 계절별 평균 검색량:")
for season, value in season_avg.items():
    print(f"  {season}: {value:.2f}")

# %%
# 겨울/여름 비율 계산
summer_avg = season_avg['여름']
winter_avg = season_avg['겨울']
ratio = winter_avg / summer_avg * 100

print(f"\n💡 핵심 지표:")
print(f"  여름 평균: {summer_avg:.2f}")
print(f"  겨울 평균: {winter_avg:.2f}")
print(f"  겨울/여름 비율: {ratio:.1f}%")

📊 계절별 평균 검색량:
  봄: 17.27
  여름: 16.29
  겨울: 8.13
  가을: 8.09

💡 핵심 지표:
  여름 평균: 16.29
  겨울 평균: 8.13
  겨울/여름 비율: 49.9%


### 📈 시각화 1: 계절별 분포

In [5]:
# Plotly 인터랙티브 박스플롯 + 바이올린 플롯
fig = make_subplots(
    rows=1, cols=2,
    subplot_titles=('계절별 박스플롯', '계절별 바이올린 플롯'),
    specs=[[{"type": "box"}, {"type": "box"}]]
)

seasons_order = ['봄', '여름', '가을', '겨울']
colors = {'봄': '#90EE90', '여름': '#FFD700', '가을': '#FF6347', '겨울': '#87CEEB'}

# 박스플롯
for season in seasons_order:
    data = df_trend[df_trend['season'] == season]['선크림']
    fig.add_trace(
        go.Box(y=data, name=season, marker_color=colors[season], boxmean='sd'),
        row=1, col=1
    )

# 바이올린 플롯
for season in seasons_order:
    data = df_trend[df_trend['season'] == season]['선크림']
    fig.add_trace(
        go.Violin(y=data, name=season, marker_color=colors[season], showlegend=False),
        row=1, col=2
    )

fig.update_yaxes(title_text="검색량 지수", row=1, col=1)
fig.update_yaxes(title_text="검색량 지수", row=1, col=2)
fig.update_layout(
    title_text="📊 계절별 선크림 검색량 분포 (인터랙티브)",
    height=500,
    showlegend=True,
    hovermode='closest'
)
fig.show()

### 🔬 통계적 유의성 검정

In [6]:

# 여름 vs 겨울 t-test
summer_data = df_trend[df_trend['season']=='여름']['선크림']
winter_data = df_trend[df_trend['season']=='겨울']['선크림']

t_stat, p_value = stats.ttest_ind(summer_data, winter_data)

print("📈 t-검정 결과:")
print(f"  t-통계량: {t_stat:.3f}")
print(f"  p-value: {p_value:.4f}")
print(f"  자유도: {len(summer_data) + len(winter_data) - 2}")

if p_value < 0.05:
    print(f"\n  ✅ 결론: 여름과 겨울의 검색량 차이는 통계적으로 유의미합니다 (p<0.05)")
else:
    print(f"\n  ⚠️ 결론: 여름과 겨울의 검색량 차이는 통계적으로 유의미하지 않습니다 (p≥0.05)")
    
# 효과 크기 (Cohen's d)
pooled_std = np.sqrt(((len(summer_data)-1)*summer_data.std()**2 + 
                      (len(winter_data)-1)*winter_data.std()**2) / 
                     (len(summer_data) + len(winter_data) - 2))
cohens_d = (summer_data.mean() - winter_data.mean()) / pooled_std

print(f"\n📏 효과 크기 (Cohen's d): {cohens_d:.3f}")
if abs(cohens_d) < 0.5:
    print("  → 작은 효과")
elif abs(cohens_d) < 0.8:
    print("  → 중간 효과")
else:
    print("  → 큰 효과")

📈 t-검정 결과:
  t-통계량: 8.321
  p-value: 0.0000
  자유도: 21

  ✅ 결론: 여름과 겨울의 검색량 차이는 통계적으로 유의미합니다 (p<0.05)

📏 효과 크기 (Cohen's d): 3.474
  → 큰 효과


### 📅 연도별 겨울 트렌드

In [7]:
# 연도별 겨울 검색량
df_winter = df_trend[df_trend['season']=='겨울'].copy()
yearly_winter = df_winter.groupby('year')['선크림'].mean()

print("📅 연도별 겨울 평균 검색량:")
for year, value in yearly_winter.items():
    print(f"  {year}년: {value:.2f}")

📅 연도별 겨울 평균 검색량:
  2022년: 8.39
  2023년: 9.19
  2024년: 7.43
  2025년: 7.16


### 🎯 인사이트 요약

In [8]:
# 증가율 계산
growth_rates = yearly_winter.pct_change() * 100
print(f"\n📈 전년 대비 증가율:")
for year, rate in growth_rates.dropna().items():
    print(f"  {year}년: {rate:+.1f}%")


📈 전년 대비 증가율:
  2023년: +9.5%
  2024년: -19.2%
  2025년: -3.6%


In [13]:
# 여름/겨울 비교 + 연도별 트렌드 (서브플롯)
fig = make_subplots(
    rows=2, cols=1,
    subplot_titles=('연도별 겨울 vs 여름 검색량 추이', '연도별 겨울 검색량 성장율'),
    specs=[[{"secondary_y": False}], [{"secondary_y": False}]],
    vertical_spacing=0.12
)

# 1번 그래프: 연도별 겨울/여름 비교
yearly_summer = df_trend[df_trend['season']=='여름'].groupby('year')['선크림'].mean()
yearly_winter = df_trend[df_trend['season']=='겨울'].groupby('year')['선크림'].mean()

fig.add_trace(
    go.Scatter(
        x=yearly_winter.index.astype(str), 
        y=yearly_winter.values,
        mode='lines+markers',
        name='겨울',
        line=dict(color='#87CEEB', width=3),
        marker=dict(size=10, symbol='circle')
    ),
    row=1, col=1
)

fig.add_trace(
    go.Scatter(
        x=yearly_summer.index.astype(str), 
        y=yearly_summer.values,
        mode='lines+markers',
        name='여름',
        line=dict(color='#FFD700', width=3),
        marker=dict(size=10, symbol='diamond')
    ),
    row=1, col=1
)

# 2번 그래프: 성장율
growth_rates = yearly_winter.pct_change() * 100
colors_growth = ['green' if x > 0 else 'red' for x in growth_rates.dropna()]

fig.add_trace(
    go.Bar(
        x=growth_rates.dropna().index.astype(str),
        y=growth_rates.dropna().values,
        marker_color=colors_growth,
        name='성장율',
        showlegend=False
    ),
    row=2, col=1
)

fig.add_hline(y=0, line_dash="dash", line_color="black", row=2, col=1)

fig.update_xaxes(title_text="연도", row=1, col=1)
fig.update_xaxes(title_text="연도", row=2, col=1)
fig.update_yaxes(title_text="검색량 지수", row=1, col=1)
fig.update_yaxes(title_text="전년 대비 증가율 (%)", row=2, col=1)

fig.update_layout(
    title_text="📈 겨울 선크림 검색량 연도별 변화 분석",
    height=700,
    hovermode='x unified'
)
fig.show()

In [12]:
print("=" * 60)
print("💡 Q1 인사이트")
print("=" * 60)

print(f"\n1️⃣ 시장 규모:")
print(f"   겨울 시장은 여름 대비 {ratio:.1f}%의 수요를 보임")

if ratio > 30:
    print(f"   ✅ 투자 가치 충분 (30% 이상)")
elif ratio > 20:
    print(f"   ⚠️ 중간 수준 시장 (20-30%)")
else:
    print(f"   ❌ 소규모 시장 (20% 미만)")

print(f"\n2️⃣ 통계적 유의성:")
print(f"   p-value = {p_value:.4f} → {'유의미' if p_value < 0.05 else '유의미하지 않음'}")

print(f"\n3️⃣ 성장 추세:")
if len(growth_rates.dropna()) > 0:
    avg_growth = growth_rates.dropna().mean()
    print(f"   연평균 증가율: {avg_growth:+.1f}%")
    if avg_growth > 0:
        print(f"   ✅ 시장 성장 중")
    else:
        print(f"   ⚠️ 시장 정체/감소")

💡 Q1 인사이트

1️⃣ 시장 규모:
   겨울 시장은 여름 대비 49.9%의 수요를 보임
   ✅ 투자 가치 충분 (30% 이상)

2️⃣ 통계적 유의성:
   p-value = 0.0000 → 유의미

3️⃣ 성장 추세:
   연평균 증가율: -4.4%
   ⚠️ 시장 정체/감소


### 🎨 추가 시각화: 월별 트렌드 및 히트맵

In [14]:
# 월별 평균 + 연도별 히트맵
fig = make_subplots(
    rows=1, cols=2,
    subplot_titles=('월별 평균 검색량', '연도-월 히트맵'),
    specs=[[{"type": "bar"}, {"type": "heatmap"}]]
)

# 1. 월별 평균
monthly_avg = df_trend.groupby('month')['선크림'].mean()
month_names = ['1월', '2월', '3월', '4월', '5월', '6월', '7월', '8월', '9월', '10월', '11월', '12월']

fig.add_trace(
    go.Bar(
        x=month_names,
        y=monthly_avg.values,
        marker_color='#FF6347',
        showlegend=False,
        name='월별 평균'
    ),
    row=1, col=1
)

# 2. 연도-월 히트맵
pivot_data = df_trend.pivot_table(values='선크림', index='year', columns='month', aggfunc='mean')

fig.add_trace(
    go.Heatmap(
        z=pivot_data.values,
        x=month_names,
        y=pivot_data.index.astype(str),
        colorscale='YlOrRd',
        colorbar=dict(title="검색량"),
        showscale=True,
        name='히트맵'
    ),
    row=1, col=2
)

fig.update_xaxes(title_text="월", row=1, col=1)
fig.update_yaxes(title_text="검색량 지수", row=1, col=1)
fig.update_xaxes(title_text="월", row=1, col=2)
fig.update_yaxes(title_text="연도", row=1, col=2)

fig.update_layout(
    title_text="📅 월별 패턴 분석 및 히트맵",
    height=500,
    width=1200
)
fig.show()

### 🎯 종합 대시보드

In [15]:
# 4개 서브플롯 대시보드
fig = make_subplots(
    rows=2, cols=2,
    subplot_titles=(
        '📊 계절별 분포',
        '📈 연도별 겨울 트렌드',
        '📅 월별 평균 검색량',
        '🔥 계절별 Box Plot'
    ),
    specs=[
        [{"type": "pie"}, {"type": "scatter"}],
        [{"type": "bar"}, {"type": "box"}]
    ],
    vertical_spacing=0.15,
    horizontal_spacing=0.12
)

# 1. 파이 차트 (계절별 비중)
season_total = df_trend.groupby('season')['선크림'].sum().reindex(['봄', '여름', '가을', '겨울'])
fig.add_trace(
    go.Pie(
        labels=season_total.index,
        values=season_total.values,
        marker=dict(colors=['#90EE90', '#FFD700', '#FF6347', '#87CEEB']),
        name='계절비중'
    ),
    row=1, col=1
)

# 2. 선 그래프 (연도별 겨울)
yearly_winter = df_trend[df_trend['season']=='겨울'].groupby('year')['선크림'].mean()
fig.add_trace(
    go.Scatter(
        x=yearly_winter.index.astype(str),
        y=yearly_winter.values,
        mode='lines+markers',
        name='겨울',
        line=dict(color='#87CEEB', width=3),
        marker=dict(size=8)
    ),
    row=1, col=2
)

# 3. 바 차트 (월별)
monthly_avg = df_trend.groupby('month')['선크림'].mean()
month_names = ['1월', '2월', '3월', '4월', '5월', '6월', '7월', '8월', '9월', '10월', '11월', '12월']
fig.add_trace(
    go.Bar(
        x=month_names,
        y=monthly_avg.values,
        marker_color='#FF6347',
        name='월별'
    ),
    row=2, col=1
)

# 4. 박스 플롯 (계절별)
for season in ['봄', '여름', '가을', '겨울']:
    data = df_trend[df_trend['season'] == season]['선크림']
    color_map = {'봄': '#90EE90', '여름': '#FFD700', '가을': '#FF6347', '겨울': '#87CEEB'}
    fig.add_trace(
        go.Box(
            y=data,
            name=season,
            marker_color=color_map[season]
        ),
        row=2, col=2
    )

fig.update_xaxes(title_text="연도", row=1, col=2)
fig.update_yaxes(title_text="검색량", row=1, col=2)
fig.update_xaxes(title_text="월", row=2, col=1)
fig.update_yaxes(title_text="검색량", row=2, col=1)

fig.update_layout(
    title_text="🎨 선크림 검색 트렌드 종합 대시보드",
    height=800,
    width=1300,
    showlegend=True,
    hovermode='closest'
)
fig.show()